<a href="https://colab.research.google.com/github/hood-boi/world-news-chatbot/blob/master/ChatbotPROD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torchtext
import random
import requests
import json
import time

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True,
                                  init_token="<BOS>",
                                  eos_token="<EOS>"
                                 )       # to turn each character into an integer index
label_field = torchtext.data.Field(sequential=True,    # text sequence
                                   use_vocab=True,     # don't need to track vocabulary
                                   is_target=True,      
                                   batch_first=True,
                                   tokenize=lambda x: x,
                                   preprocessing=lambda x: x,
                                   init_token="<BOS>",
                                   eos_token="<EOS>"
                                  ) 

fields = [('reply', label_field), ('context', text_field)]
dataset1 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rCasualConversation.txt", "tsv", fields); # name of the file
dataset2 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rAskReddit.txt", "tsv", fields); # name of the file
dataset3 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rnews.txt", "tsv", fields); # name of the file
dataset4 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rworldnews.txt", "tsv", fields); # name of the file
dataset5 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/routoftheloop.txt", "tsv", fields); # name of the file
dataset6 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rnottheonion.txt", "tsv", fields); # name of the file
#dataset7 = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rCasualConversation.txt", "tsv", fields); # name of the file
dataset = torch.utils.data.ConcatDataset([dataset1, dataset2, dataset3, dataset4, dataset5, dataset6])

In [0]:
train = torchtext.data.Dataset(dataset, fields)
text_field.build_vocab(train)
label_field.build_vocab(train)
print(text_field.vocab.itos)
print(label_field.vocab.itos)
input_vocab_size = len(text_field.vocab.itos)
reply_vocab_size = len(label_field.vocab.itos)
print("Input Vocab Size: ", input_vocab_size)
print("Reply Vocab size: ", reply_vocab_size)

['<unk>', '<pad>', '<BOS>', '<EOS>', ' ', 'e', 't', 'a', 'o', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'c', 'm', 'y', 'g', 'p', 'f', 'w', 'b', '.', 'k', 'v', 'I', ',', 'T', 'S', 'A', '?', 'W', 'C', '0', '-', '/', 'M', 'x', 'j', 'P', 'H', 'D', 'B', 'N', 'E', 'R', ':', '1', 'F', 'O', '2', 'z', 'G', '!', 'L', 'U', 'Y', '*', 'J', '5', 'K', ')', 'q', '(', '3', '4', '9', ';', '8', '6', '7', 'V', '$', '_', ']', '[', '%', '=', '&', '^', '~', 'Z', 'Q', 'X', '#', '+', '\\', '|', '@', '`', '}', '{']
['<unk>', '<pad>', '<BOS>', '<EOS>', ' ', 'e', 't', 'o', 'a', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'm', 'c', 'y', 'g', 'w', 'p', 'f', '.', 'b', 'k', 'v', 'I', ',', 'T', 'A', 'S', '/', '?', 'j', 'W', 'x', '-', 'H', '!', '0', 'C', 'M', 'N', 'E', 'B', 'O', 'D', 'P', '1', 'Y', 'R', ':', 'z', 'F', '*', 'L', 'G', '2', ')', '(', 'U', 'q', ';', '5', 'J', '3', 'K', '4', '9', '_', '8', '6', '7', 'V', ']', '[', '$', '^', '%', '=', '&', '~', 'Q', '#', 'Z', 'X', '+', '\\', '@', '|', '`', '{', '}']
Input Vocab Si

In [0]:
class ChatBot(nn.Module):
    def __init__(self, 
                 context_vocab_size,  
                 reply_vocab_size, 
                 encoder_hidden_size = 100,
                 generator_hidden_size = 100, 
                 encoder_layers = 1, 
                 generator_layers = 1):
        
        super(ChatBot, self).__init__()
        
        self.encoder_layers = encoder_layers;
        self.generator_layers =generator_layers;
        self.encoder_hidden_size = encoder_hidden_size;
        self.generator_hidden_size = generator_hidden_size;
        
        # >>> Encoder
        self.context_ident = torch.eye(context_vocab_size)
        self.encode_rnn = nn.LSTM(context_vocab_size, encoder_hidden_size, encoder_layers, batch_first=True)
        
        # >>> Generator
        self.reply_ident = torch.eye(reply_vocab_size)
        self.decode_rnn = nn.LSTM(reply_vocab_size, generator_hidden_size, generator_layers, batch_first=True)
        self.fcnn = nn.Linear(generator_hidden_size, reply_vocab_size)
        
    def forward(self, context, response, hidden=None):
        
        # >>> Encoder
        context_tensor = self.context_ident[context] # Type: batch.context[0] | Size: (batch size, sequence size)
        encode_out, encode_last_hidden = self.encode_rnn(context_tensor)
        
        # >>> Generator
        reply_tensor = self.reply_ident[response] #Type: batch.reply
        if(hidden == None):
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, encode_last_hidden)
        else:
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, hidden)
        out = self.fcnn(gen_out)
        return out, gen_last_hidden

In [0]:
model = ChatBot(input_vocab_size, reply_vocab_size, encoder_layers = 2, generator_layers = 2)
model = model.cuda();
model.context_ident = model.context_ident.cuda();
model.reply_ident = model.reply_ident.cuda();

In [0]:
len(dataset)

1499611

In [0]:
def train_fcn(model, data, reply_vocab_size, batch_size=1, num_epochs=1, lr=0.001, print_every=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    it = 0
    
    data_iter = torchtext.data.BucketIterator(data,
                                              batch_size=batch_size,
                                              sort_key=lambda x: len(x.context),
                                              sort_within_batch=True)
    print("Here #1 : ")
    for e in range(num_epochs):
        torch.save(model.state_dict(), 'weights_'+str(e))
        print("Epoch : ", e);
        # get training set
        avg_loss = 0
        for batch in data_iter:
            optimizer.zero_grad();
            inp = batch.context[0] # BOS + EOS
            inp = inp.cuda()
            reply = batch.reply[:, :-1]
            reply = reply.cuda()
            
            out, hidden = model(inp, reply) # no eos
            out_reshaped = out.reshape(-1,reply_vocab_size)
            target = (batch.reply[:,1:]).reshape(-1) # no bos
            target = target.cuda()
            #print(batch.reply[:,1:])
            loss = criterion(out_reshaped, target)
            
            loss.backward();
            optimizer.step();
            
            avg_loss += loss
            it += 1 # increment iteration count
            if it % print_every == 0:
                print("[Iter %d] Loss %f" % (it+1, float(avg_loss/print_every)))
                avg_loss = 0
        torch.save(model.state_dict(), 'weights_'+str(e))

In [0]:
train_fcn(model, train, reply_vocab_size, batch_size =128, num_epochs = 10)

In [0]:
def sample_sequence(model, context, text_field, label_field, max_len=100, temperature=0.5):
    generated_sequence = ""
    
    data_inp = ["<BOS>"] + list(context) + ["<EOS>"];
    data_inp_indices = [text_field.vocab.stoi[ch] for ch in data_inp];
    data_inp_tensor = torch.Tensor(data_inp_indices).long().unsqueeze(0);
    
    inp = torch.Tensor([label_field.vocab.stoi["<BOS>"]]).long()
    hidden = None;
    for p in range(max_len):
        #print(inp)
        output, hidden = model(data_inp_tensor, inp.unsqueeze(0), hidden)
        #print(output)
        #output = F.softmax(output, dim=2)
        #print(torch.argmax(output, dim=2))
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        #print(output_dist)
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = label_field.vocab.itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence

In [0]:
test_model = ChatBot(input_vocab_size, reply_vocab_size, encoder_layers = 2, generator_layers = 2)
test_model.load_state_dict(torch.load('weights_2'))
test_model.eval()

ChatBot(
  (encode_rnn): LSTM(95, 100, num_layers=2, batch_first=True)
  (decode_rnn): LSTM(95, 100, num_layers=2, batch_first=True)
  (fcnn): Linear(in_features=100, out_features=95, bias=True)
)

In [0]:
sample_sequence(test_model, "What is net neutrality?", text_field, label_field)


'Im a little some people are with the property of the country that would have seen a better comment t'